<a href="https://colab.research.google.com/github/Ramanaa-Vijayakumar/Smart-Product-Labeling-and-Traceability-System/blob/main/Smart_Labeling_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install dependencies if not already
!pip install opencv-python numpy pandas easyocr reportlab qrcode[pil] scikit-learn pygame

import random
import time
import sqlite3
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas
import qrcode
from PIL import Image

# ----------------------------
# Product Generator
# ----------------------------
def generate_product():
    return {
        "device_id": f"DEV-{random.randint(1000, 9999)}",
        "batch_id": f"BATCH-{random.randint(1, 5)}",
        "manufacturing_date": f"2023-{random.randint(1, 12):02d}-{random.randint(1, 28):02d}",
        "rohs_compliant": random.choice([True, False]),
        "has_defect": random.random() < 0.1  # 10% defect rate
    }

# ----------------------------
# Label Generator
# ----------------------------
def generate_label_pdf(product):
    filename = f"label_{product['device_id']}.pdf"
    c = canvas.Canvas(filename, pagesize=(100, 60))

    # Add text
    c.setFont("Helvetica", 6)
    c.drawString(5, 50, f"Device: {product['device_id']}")
    c.drawString(5, 40, f"Batch: {product['batch_id']}")
    c.drawString(5, 30, f"MFG Date: {product['manufacturing_date']}")
    c.drawString(5, 20, f"RoHS: {'✓' if product['rohs_compliant'] else '✗'}")

    # Add QR code
    qr = qrcode.QRCode(version=1, box_size=2, border=2)
    qr.add_data(product['device_id'])
    qr.make(fit=True)
    qr_img = qr.make_image(fill_color="black", back_color="white")
    qr_img.save("temp_qr.png")
    c.drawImage("temp_qr.png", 60, 5, width=35, height=35)

    c.save()
    return filename

# ----------------------------
# Dummy Label Scanner (simulated)
# ----------------------------
def scan_label(pdf_file):
    # Simulate label scanning by extracting device ID from filename
    if "label_DEV-" in pdf_file:
        return pdf_file.split("label_")[1].replace(".pdf", "")
    else:
        return "UNKNOWN"

# ----------------------------
# Simulated Defect Checker
# ----------------------------
def check_for_defects(product):
    if product["has_defect"]:
        return {"status": "REJECT", "reason": "Product defect detected"}
    else:
        return {"status": "ACCEPT", "reason": "No defects found"}

# ----------------------------
# SQLite DB Setup
# ----------------------------
conn = sqlite3.connect("product_traceability.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS products (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    device_id TEXT NOT NULL,
    batch_id TEXT NOT NULL,
    manufacturing_date TEXT NOT NULL,
    rohs_compliant INTEGER NOT NULL,
    inspection_result TEXT NOT NULL,
    inspection_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")
conn.commit()

# ----------------------------
# Logging Function
# ----------------------------
def log_product(product, result):
    cursor.execute("""
    INSERT INTO products (device_id, batch_id, manufacturing_date, rohs_compliant, inspection_result)
    VALUES (?, ?, ?, ?, ?)
    """, (
        product["device_id"],
        product["batch_id"],
        product["manufacturing_date"],
        int(product["rohs_compliant"]),
        result["status"]
    ))
    conn.commit()

# ----------------------------
# Conveyor Simulation
# ----------------------------
def run_full_simulation():
    print("🔄 Starting full simulation...\n")
    for _ in range(10):  # Simulate 10 products
        product = generate_product()
        print(f"🧾 Processing: {product['device_id']}")

        # Step 1: Generate label
        label_pdf = generate_label_pdf(product)

        # Step 2: Scan label
        scanned_data = scan_label(label_pdf)
        print(f"🔍 Scanned Device ID: {scanned_data}")

        # Step 3: Defect check
        defect_result = check_for_defects(product)
        print(f"🧪 Inspection Result: {defect_result['status']} ({defect_result['reason']})")

        # Step 4: Log to database
        log_product(product, defect_result)
        print("✅ Logged to database.\n")

        time.sleep(1)  # Simulate delay

    print("✅✅ Simulation completed for 10 products.\n")

# ----------------------------
# Run it!
# ----------------------------
run_full_simulation()


🔄 Starting full simulation...

🧾 Processing: DEV-7584
🔍 Scanned Device ID: DEV-7584
🧪 Inspection Result: ACCEPT (No defects found)
✅ Logged to database.

🧾 Processing: DEV-5838
🔍 Scanned Device ID: DEV-5838
🧪 Inspection Result: ACCEPT (No defects found)
✅ Logged to database.

🧾 Processing: DEV-6783
🔍 Scanned Device ID: DEV-6783
🧪 Inspection Result: REJECT (Product defect detected)
✅ Logged to database.

🧾 Processing: DEV-5948
🔍 Scanned Device ID: DEV-5948
🧪 Inspection Result: REJECT (Product defect detected)
✅ Logged to database.

🧾 Processing: DEV-6869
🔍 Scanned Device ID: DEV-6869
🧪 Inspection Result: ACCEPT (No defects found)
✅ Logged to database.

🧾 Processing: DEV-2878
🔍 Scanned Device ID: DEV-2878
🧪 Inspection Result: ACCEPT (No defects found)
✅ Logged to database.

🧾 Processing: DEV-6479
🔍 Scanned Device ID: DEV-6479
🧪 Inspection Result: ACCEPT (No defects found)
✅ Logged to database.

🧾 Processing: DEV-4161
🔍 Scanned Device ID: DEV-4161
🧪 Inspection Result: ACCEPT (No defects f